# Player performance until the moment of the transfer

In [1]:
import pandas as pd
import numpy as np
import sys 
import csv
import unicodedata
import datetime as dt
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

We need the date in the dataset about transfers in the format %y-%m-%d so that we can compare it with the data of each match we will find in the dataset of each player

In [7]:
d_tf = pd.read_csv('data_17.csv')

In [8]:
d_tf = d_tf.drop(['Unnamed: 0'], axis=1)

In [9]:
def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

def remove_special (input_str):
    new = input_str.replace("'", "").replace("æ", "ae").replace("Ð", "D").replace("ð", "d").replace("Ø", "O").replace("ø", "o").replace("Æ", "AE").replace("œ", "oe").replace('ς','c').replace('đ','d')
    return new

In [10]:
d_tf['name_final'] = d_tf.shortName.apply(remove_accents).apply(remove_special)

# STATISTICS

NB: non prendere i portieri perché hanno statistiche tutte diverse!!
T=total, S=successfull

In [15]:
d_tf = d_tf[d_tf.role != 'Goalkeeper']

In [16]:
d_tf.shape[0]

772

In [17]:
def renameColumns(df):
    return df.rename(columns={'Minutes played':'minutes',
                   'Total actions / successful':'t_actions',
                   'Unnamed: 6':'s_actions',
                   'Shots / on target':'t_shots',
                   'Unnamed: 10':'shots_on_target',
                   'Passes / accurate':'t_passes',
                   'Unnamed: 13':'s_passes',
                   'Long passes / accurate':'t_longpasses',
                   'Unnamed: 15':'s_longpasses',
                   'Crosses / accurate':'t_crosses',
                   'Unnamed: 17':'s_crosses', 
                   'Dribbles / successful':'t_dribb',
                   'Unnamed: 19':'s_dribb', 
                   'Duels / won':'t_duels', 
                   'Unnamed: 21':'s_duels', 
                   'Aerial duels / won':'t_aerialduels',
                   'Unnamed: 23':'s_aerialduels', 
                   'Losses / own half':'t_losses', 
                   'Unnamed: 26':'bad_losses',
                   'Recoveries / opp. half':'t_recovers', 
                   'Unnamed: 28':'good_recovers', 
                   'Yellow card':'yellow', 
                   'Red card':'red'
                  })

In [18]:
def keepBeforeDate(df,date):
    date_compare = pd.to_datetime(date)
    date_format = '%Y-%m-%d'
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] < date_compare]
    return df

def keepAfterDate(df,date):
    date_compare = pd.to_datetime(date)
    date_format = '%Y-%m-%d'
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] >= date_compare]
    return df

Functions to calculate players' statistics

In [19]:
def minutesFun(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        totalMinutes = n_matches*90
        sum_col = df.sum(axis = 0, skipna = True) 
        minutes=0
        if(round(sum_col.minutes/totalMinutes*100)>90):
            minutes=90
        else:
            minutes=round(sum_col.minutes/totalMinutes*100)
    else:
        minutes=0
    return minutes

def percSuccessfulActionsFun(df):
    if(sum(df.t_actions) != 0):
        perc_s_actions = round(sum(df.s_actions)/sum(df.t_actions)*100,2)
    else:
        perc_s_actions = 0
    return perc_s_actions

def goalsPerMatch(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        goal_per_match = round(sum(df.Goals)/n_matches,3)
    else:
        goal_per_match = 0
    return goal_per_match

def assistsPerMatch(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        assist_per_match = round(sum(df.Assists)/n_matches,3)
    else:
        assist_per_match = 0
    return assist_per_match

def percShotsOnTarget(df):
    if(sum(df.t_shots) != 0):
        perc_shots_target = round(sum(df.shots_on_target)/sum(df.t_shots)*100,2)
    else:
        perc_shots_target = 0
    return perc_shots_target

def ExpectedGoals(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        xg = round(sum(df.xG)/n_matches,3)
    else:
        xg=0
    return xg

def passesAccuracy(df):
    if(sum(df.t_passes) != 0):
        passes_acc = round(sum(df.s_passes)/sum(df.t_passes)*100,2) 
    else:
        passes_acc = 0
    return passes_acc

def longPassesAccuracy(df):
    if(sum(df.t_longpasses) != 0):
        lpasses_acc = round(sum(df.s_longpasses)/sum(df.t_longpasses)*100,2)
    else:
        lpasses_acc = 0
    return lpasses_acc

def crossesAccuracy(df):
    if(sum(df.t_crosses) != 0):
        crosses_acc = round(sum(df.s_crosses)/sum(df.t_crosses)*100,2)
    else:
        crosses_acc = 0
    return crosses_acc

def percSuccessfulDribb(df):
    if(sum(df.t_dribb) != 0):
        perc_dribbles = round(sum(df.s_dribb)/sum(df.t_dribb)*100,2)
    else:
        perc_dribbles = 0
    return perc_dribbles

def percSuccessfulDuels(df):
    if(sum(df.t_duels) != 0):
        perc_duels = round(sum(df.s_duels)/sum(df.t_duels)*100,2)
    else:
        perc_duels = 0
    return perc_duels

def percAerialDuels(df):
    if(sum(df.t_aerialduels) != 0):
        perc_aerialduels = round(sum(df.s_aerialduels)/sum(df.t_aerialduels)*100,2)
    else:
        perc_aerialduels = 0
    return perc_aerialduels

def numberInterceptions(df):
    n_matches = df.shape[0]
    sum_col = df.sum(axis = 0, skipna = True) 
    if(n_matches != 0):
        interceptions = round(sum_col.Interceptions/n_matches,2)
    else:
        interceptions = 0
    return interceptions

def recoveriesOpposite(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        sum_col = df.sum(axis = 0, skipna = True) 
        rec_opposite = round(sum_col.good_recovers/n_matches,2)
    else:
        rec_opposite = 0
    return rec_opposite   

def recoveriesTot(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        sum_col = df.sum(axis = 0, skipna = True) 
        rec_tot = round(sum_col.t_recovers/n_matches,2)
    else:
        rec_tot = 0
    return rec_tot   

def lossesOwn(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        sum_col = df.sum(axis = 0, skipna = True) 
        los_own = round(sum_col.bad_losses/n_matches,2)
    else:
        los_own = 0
    return los_own   

def lossesTotal(df):
    n_matches = df.shape[0]
    if(n_matches != 0):
        sum_col = df.sum(axis = 0, skipna = True) 
        los_tot = round(sum_col.t_losses/n_matches,2)
    else:
        los_tot = 0
    return los_tot

def numberYellow(df):
    yellow = len(df[df['yellow'] != 0])
    return yellow 

def numberRed(df):
    red = len(df[df['red'] != 0])
    return red 

#def percBadLosses(df):
#    sum_col = df.sum(axis = 0, skipna = True) 
 #   perc_losses_bad = round((sum_col.bad_losses/sum_col.t_losses)*100,1)
  #  return perc_losses_bad

In [20]:
# empty columns
d_tf['number_matches'] = np.nan
d_tf['minutes'] = np.nan
d_tf['perc_s_actions'] = np.nan
d_tf['goal_per_match'] = np.nan
d_tf['assist_per_match'] = np.nan
d_tf['perc_shots_target'] = np.nan
d_tf['xg'] = np.nan
d_tf['passes_acc'] = np.nan
d_tf['lpasses_acc'] = np.nan 
d_tf['crosses_acc'] = np.nan
d_tf['perc_dribbles'] = np.nan
d_tf['perc_duels'] = np.nan
d_tf['perc_aerialduels'] = np.nan
d_tf['interceptions'] = np.nan
d_tf['rec_opposite'] = np.nan
d_tf['rec_tot'] = np.nan
d_tf['los_own'] = np.nan
d_tf['los_tot'] = np.nan
d_tf['yellow'] = np.nan
d_tf['red'] = np.nan

In [21]:
# 1 year before
d_tf['number_matches1Year'] = np.nan
d_tf['minutes1Year'] = np.nan
d_tf['perc_s_actions1Year'] = np.nan
d_tf['goal_per_match1Year'] = np.nan
d_tf['assist_per_match1Year'] = np.nan
d_tf['perc_shots_target1Year'] = np.nan
d_tf['xg1Year'] = np.nan
d_tf['passes_acc1Year'] = np.nan
d_tf['lpasses_acc1Year'] = np.nan 
d_tf['crosses_acc1Year'] = np.nan
d_tf['perc_dribbles1Year'] = np.nan
d_tf['perc_duels1Year'] = np.nan
d_tf['perc_aerialduels1Year'] = np.nan
d_tf['interceptions1Year'] = np.nan
d_tf['rec_opposite1Year'] = np.nan
d_tf['rec_tot1Year'] = np.nan
d_tf['los_own1Year'] = np.nan
d_tf['los_tot1Year'] = np.nan
d_tf['yellow1Year'] = np.nan
d_tf['red1Year'] = np.nan


In [22]:
# 6 months before
d_tf['number_matches6Months'] = np.nan
d_tf['minutes6Months'] = np.nan
d_tf['perc_s_actions6Months'] = np.nan
d_tf['goal_per_match6Months'] = np.nan
d_tf['assist_per_match6Months'] = np.nan
d_tf['perc_shots_target6Months'] = np.nan
d_tf['xg6Months'] = np.nan
d_tf['passes_acc6Months'] = np.nan
d_tf['lpasses_acc6Months'] = np.nan 
d_tf['crosses_acc6Months'] = np.nan
d_tf['perc_dribbles6Months'] = np.nan
d_tf['perc_duels6Months'] = np.nan
d_tf['perc_aerialduels6Months'] = np.nan
d_tf['interceptions6Months'] = np.nan
d_tf['rec_opposite6Months'] = np.nan
d_tf['rec_tot6Months'] = np.nan
d_tf['los_own6Months'] = np.nan
d_tf['los_tot6Months'] = np.nan
d_tf['yellow6Months'] = np.nan
d_tf['red6Months'] = np.nan


In [23]:
for i,row in d_tf.iterrows():
    date=row.date
    name=row.name_final
    
    df1 = pd.read_excel("./player_performance/Player Stats " + name + ".xlsx",encoding = "utf8") # da generalizzare
    
    # 1 Rename columns of the dataset
    df1 = renameColumns(df1)
    
    # 2 Consider data only the date of the transfer
    df1 = keepBeforeDate(df1, date)
    
    # 3 STATISTICS
    minutes = minutesFun(df1)
    perc_s_actions = percSuccessfulActionsFun(df1)
    goal_per_match = goalsPerMatch(df1)
    assist_per_match = assistsPerMatch(df1)
    perc_shots_target = percShotsOnTarget(df1)
    xg = ExpectedGoals(df1)    
    passes_acc = passesAccuracy(df1)
    lpasses_acc = longPassesAccuracy(df1)
    crosses_acc = crossesAccuracy(df1)
    perc_dribbles = percSuccessfulDribb(df1)
    perc_duels = percSuccessfulDuels(df1)
    perc_aerialduels = percAerialDuels(df1)
    interceptions = numberInterceptions(df1)
    rec_opposite = recoveriesOpposite(df1)
    rec_tot = recoveriesTot(df1)
    los_own = lossesOwn(df1)
    los_tot = lossesTotal(df1)
    yellow = numberYellow(df1)
    red = numberRed(df1)
    n_matches = df1.shape[0]
    
    d_tf.loc[i,'number_matches'] = n_matches    
    d_tf.loc[i,'minutes'] = minutes
    d_tf.loc[i,'perc_s_actions'] = perc_s_actions
    d_tf.loc[i,'goal_per_match'] = goal_per_match
    d_tf.loc[i,'assist_per_match'] = assist_per_match
    d_tf.loc[i,'perc_shots_target'] = perc_shots_target
    d_tf.loc[i,'xg'] = xg
    d_tf.loc[i,'passes_acc'] = passes_acc
    d_tf.loc[i,'lpasses_acc'] = lpasses_acc
    d_tf.loc[i,'crosses_acc'] = crosses_acc
    d_tf.loc[i,'perc_dribbles'] = perc_dribbles
    d_tf.loc[i,'perc_duels'] = perc_duels
    d_tf.loc[i,'perc_aerialduels'] = perc_aerialduels
    d_tf.loc[i,'interceptions'] = interceptions
    d_tf.loc[i,'rec_opposite'] = rec_opposite
    d_tf.loc[i,'rec_tot'] = rec_tot
    d_tf.loc[i,'los_own'] = los_own
    d_tf.loc[i,'los_tot'] = los_tot
    d_tf.loc[i,'yellow'] = yellow
    d_tf.loc[i,'red'] = red
    
    # 4 find 2 dates corresponding to 6 month and 1 year before the transfer 
    date = pd.to_datetime(date)
    sixMonthsBefore = date + relativedelta(months= -6)
    oneYearBefore = date + relativedelta(months= -12)

    
    # keep only between one years before the transfer and the date of the trasfer
    df1 = keepAfterDate(df1, oneYearBefore)
    
    # 3 STATISTICS 1 Year
    minutes = minutesFun(df1)
    perc_s_actions = percSuccessfulActionsFun(df1)
    goal_per_match = goalsPerMatch(df1)
    assist_per_match = assistsPerMatch(df1)
    perc_shots_target = percShotsOnTarget(df1)
    xg = ExpectedGoals(df1)    
    passes_acc = passesAccuracy(df1)
    lpasses_acc = longPassesAccuracy(df1)
    crosses_acc = crossesAccuracy(df1)
    perc_dribbles = percSuccessfulDribb(df1)
    perc_duels = percSuccessfulDuels(df1)
    perc_aerialduels = percAerialDuels(df1)
    interceptions = numberInterceptions(df1)
    rec_opposite = recoveriesOpposite(df1)
    rec_tot = recoveriesTot(df1)
    los_own = lossesOwn(df1)
    los_tot = lossesTotal(df1)
    yellow = numberYellow(df1)
    red = numberRed(df1)
    n_matches = df1.shape[0]
    
    d_tf.loc[i,'number_matches1Year'] = n_matches    
    d_tf.loc[i,'minutes1Year'] = minutes
    d_tf.loc[i,'perc_s_actions1Year'] = perc_s_actions
    d_tf.loc[i,'goal_per_match1Year'] = goal_per_match
    d_tf.loc[i,'assist_per_match1Year'] = assist_per_match
    d_tf.loc[i,'perc_shots_target1Year'] = perc_shots_target
    d_tf.loc[i,'xg1Year'] = xg
    d_tf.loc[i,'passes_acc1Year'] = passes_acc
    d_tf.loc[i,'lpasses_acc1Year'] = lpasses_acc
    d_tf.loc[i,'crosses_acc1Year'] = crosses_acc
    d_tf.loc[i,'perc_dribbles1Year'] = perc_dribbles
    d_tf.loc[i,'perc_duels1Year'] = perc_duels
    d_tf.loc[i,'perc_aerialduels1Year'] = perc_aerialduels
    d_tf.loc[i,'interceptions1Year'] = interceptions
    d_tf.loc[i,'rec_opposite1Year'] = rec_opposite
    d_tf.loc[i,'rec_tot1Year'] = rec_tot
    d_tf.loc[i,'los_own1Year'] = los_own
    d_tf.loc[i,'los_tot1Year'] = los_tot
    d_tf.loc[i,'yellow1Year'] = yellow
    d_tf.loc[i,'red1Year'] = red

    # 6 months before the transfer
    df1 = keepAfterDate(df1, sixMonthsBefore)
    
    # 3 STATISTICS 6 months
    minutes = minutesFun(df1)
    perc_s_actions = percSuccessfulActionsFun(df1)
    goal_per_match = goalsPerMatch(df1)
    assist_per_match = assistsPerMatch(df1)
    perc_shots_target = percShotsOnTarget(df1)
    xg = ExpectedGoals(df1)    
    passes_acc = passesAccuracy(df1)
    lpasses_acc = longPassesAccuracy(df1)
    crosses_acc = crossesAccuracy(df1)
    perc_dribbles = percSuccessfulDribb(df1)
    perc_duels = percSuccessfulDuels(df1)
    perc_aerialduels = percAerialDuels(df1)
    interceptions = numberInterceptions(df1)
    rec_opposite = recoveriesOpposite(df1)
    rec_tot = recoveriesTot(df1)
    los_own = lossesOwn(df1)
    los_tot = lossesTotal(df1)
    yellow = numberYellow(df1)
    red = numberRed(df1)
    n_matches = df1.shape[0]
    
    d_tf.loc[i,'number_matches6Months'] = n_matches    
    d_tf.loc[i,'minutes6Months'] = minutes
    d_tf.loc[i,'perc_s_actions6Months'] = perc_s_actions
    d_tf.loc[i,'goal_per_match6Months'] = goal_per_match
    d_tf.loc[i,'assist_per_match6Months'] = assist_per_match
    d_tf.loc[i,'perc_shots_target6Months'] = perc_shots_target
    d_tf.loc[i,'xg6Months'] = xg
    d_tf.loc[i,'passes_acc6Months'] = passes_acc
    d_tf.loc[i,'lpasses_acc6Months'] = lpasses_acc
    d_tf.loc[i,'crosses_acc6Months'] = crosses_acc
    d_tf.loc[i,'perc_dribbles6Months'] = perc_dribbles
    d_tf.loc[i,'perc_duels6Months'] = perc_duels
    d_tf.loc[i,'perc_aerialduels6Months'] = perc_aerialduels
    d_tf.loc[i,'interceptions6Months'] = interceptions
    d_tf.loc[i,'rec_opposite6Months'] = rec_opposite
    d_tf.loc[i,'rec_tot6Months'] = rec_tot
    d_tf.loc[i,'los_own6Months'] = los_own
    d_tf.loc[i,'los_tot6Months'] = los_tot
    d_tf.loc[i,'yellow6Months'] = yellow
    d_tf.loc[i,'red6Months'] = red
    
   # if i==777:
   #     break


In [24]:
d_tf.to_csv('data_18.csv',sep=',',header=True)